In [1]:
# ================================
# EX4: Word2Vec Skip-Gram (SGNS) — Setup & Preprocessing
# ================================
import os, io, math, random, zipfile, urllib.request, numpy as np
from collections import Counter

random.seed(42)
np.random.seed(42)

# ---- Assignment-aligned hyperparams (patched)
EMBED_DIM     = 300          # 300-d embeddings
WINDOW_MAX    = 5            # dynamic window in [1, 5]
NEGATIVE_K    = 15           # stronger negative sampling (5–20 recommended)
SUBSAMPLE_T   = 1e-5         # subsampling threshold
INIT_LR       = 0.020        # slightly cooler start; we’ll decay token-wise
EPOCHS        = 3            # more than 1 epoch for stability/quality
VOCAB_MIN_CNT = 5            # prune ultra-rare tokens
CORPUS_TOKENS = None         # use full text8 (~17M tokens)
TABLE_SIZE    = int(2e6)     # larger negative sampler table for smoother draws

# ---- Download text8 (~31MB zipped)
url = "http://mattmahoney.net/dc/text8.zip"
local_zip = "text8.zip"
if not os.path.exists(local_zip):
    print("Downloading text8.zip...")
    urllib.request.urlretrieve(url, local_zip)
    print("Downloaded.")

# Read tokens
with zipfile.ZipFile(local_zip) as zf:
    with zf.open("text8") as f:
        text_bytes = f.read()
tokens = io.BytesIO(text_bytes).read().decode("utf-8").split()

if CORPUS_TOKENS is not None:
    tokens = tokens[:CORPUS_TOKENS]
print(f"Total tokens used: {len(tokens):,}")

# ---- Build vocabulary and corpus ids
freq = Counter(tokens)
vocab = {w: c for w, c in freq.items() if c >= VOCAB_MIN_CNT}
vocab_list = sorted(vocab.keys())
word2id = {w: i for i, w in enumerate(vocab_list)}
id2word = np.array(vocab_list)
V = len(word2id)
print(f"Vocab size (min_count={VOCAB_MIN_CNT}): {V:,}")

corpus = np.array([word2id[w] for w in tokens if w in word2id], dtype=np.int32)
counts = np.bincount(corpus, minlength=V)
total_count = counts.sum()
freqs = counts / total_count

# ---- Subsampling (patched to canonical word2vec form)
# p_keep(w) = (sqrt(f/t) + 1) * (t/f)
t = SUBSAMPLE_T
f = freqs + 1e-12
p_keep = (np.sqrt(f / t) + 1.0) * (t / f)
p_keep = np.clip(p_keep, 0.0, 1.0)

rand = np.random.rand(corpus.shape[0])
kept_mask = rand < p_keep[corpus]
corpus = corpus[kept_mask]
print(f"After subsampling: {len(corpus):,} tokens kept")

# ---- Negative sampling distribution (unigram^0.75)
NEG_POWER = 0.75
neg_dist = np.power(counts, NEG_POWER)
neg_dist = neg_dist / neg_dist.sum()

# Build negative table via inverse CDF sampling
cum_probs = np.cumsum(neg_dist)
u = np.random.rand(TABLE_SIZE)
neg_table = np.searchsorted(cum_probs, u).astype(np.int32)

print("Prep complete.")


Downloaded.
Total tokens used: 17,005,207
Vocab size (min_count=5): 71,290
After subsampling: 5,567,497 tokens kept
Prep complete.


In [2]:
# ================================
# EX4: SGNS Training (NumPy)
# ================================
import time

D = EMBED_DIM

# Input (target) and Output (context) embeddings
W_in  = (np.random.rand(V, D) - 0.5) / D
W_out = (np.random.rand(V, D) - 0.5) / D

def sigmoid(x):
    x = np.clip(x, -8, 8)       # stability
    return 1.0 / (1.0 + np.exp(-x))

def sample_negatives(K):
    idx = np.random.randint(0, len(neg_table), size=K)
    return neg_table[idx]

def train_one_epoch(corpus_ids, lr0, window_max=5, negatives=5, report_every=500_000):
    n = len(corpus_ids)
    t0 = time.time()
    pairs = 0

    for i, center in enumerate(corpus_ids):
        # token-wise linear learning-rate decay across the epoch
        progress = i / max(1, n)
        lr = lr0 * (1.0 - progress)
        lr = max(lr, lr0 * 0.0001)  # tiny floor to avoid freezing

        # dynamic window
        win = np.random.randint(1, window_max + 1)
        start = max(0, i - win)
        end   = min(n, i + win + 1)

        for j in range(start, end):
            if j == i:
                continue
            context = corpus_ids[j]

            v_w = W_in[center]     # (D,)
            v_c = W_out[context]   # (D,)

            # ----- positive term: log σ(vw·vc)
            score_pos = np.dot(v_w, v_c)
            grad_pos = (1.0 - sigmoid(score_pos))  # dL/d(score)

            # updates
            W_in[center]  += lr * grad_pos * v_c
            W_out[context]+= lr * grad_pos * v_w

            # ----- negative terms: sum log σ(-vw·vn)
            neg_ids = sample_negatives(negatives)
            v_ns = W_out[neg_ids]                  # (K, D)
            scores_neg = -np.dot(v_ns, v_w)        # -vw·vn
            grad_negs = (1.0 - sigmoid(scores_neg))  # (K,)

            # updates
            W_in[center]  -= lr * np.dot(grad_negs, v_ns)          # -lr * Σ grad_k * v_nk
            W_out[neg_ids] -= (lr * grad_negs[:, None]) * v_w[None, :]

            # light clipping for stability (optional but helpful)
            np.clip(W_in[center],   -0.5, 0.5, out=W_in[center])
            np.clip(W_out[context], -0.5, 0.5, out=W_out[context])
            np.clip(W_out[neg_ids], -0.5, 0.5, out=W_out[neg_ids])

            pairs += 1

        if (i + 1) % report_every == 0:
            tok_per_s = (i + 1) / (time.time() - t0 + 1e-9)
            print(f"[progress] seen {i+1:,}/{n:,} tokens | ~{tok_per_s:,.0f} tok/s")

    elapsed = time.time() - t0
    print(f"Epoch done: processed {pairs:,} (center,context) pairs in {elapsed:.1f}s")

# ---- Train for multiple epochs with shuffle each pass
for ep in range(EPOCHS):
    order = np.random.permutation(len(corpus))
    corpus_shuf = corpus[order]
    lr0 = INIT_LR * (1 - ep / max(1, EPOCHS))  # epoch-level mild decay
    print(f"\n=== Epoch {ep+1}/{EPOCHS} | base_lr={lr0:.5f} ===")
    train_one_epoch(corpus_shuf, lr0, window_max=WINDOW_MAX, negatives=NEGATIVE_K, report_every=500_000)

print("\nTraining finished. Embeddings in W_in (target) / W_out (context).")



=== Epoch 1/3 | base_lr=0.02000 ===
[progress] seen 500,000/5,567,497 tokens | ~1,386 tok/s
[progress] seen 1,000,000/5,567,497 tokens | ~1,400 tok/s
[progress] seen 1,500,000/5,567,497 tokens | ~1,415 tok/s
[progress] seen 2,000,000/5,567,497 tokens | ~1,422 tok/s
[progress] seen 2,500,000/5,567,497 tokens | ~1,425 tok/s
[progress] seen 3,000,000/5,567,497 tokens | ~1,429 tok/s
[progress] seen 3,500,000/5,567,497 tokens | ~1,435 tok/s
[progress] seen 4,000,000/5,567,497 tokens | ~1,433 tok/s
[progress] seen 4,500,000/5,567,497 tokens | ~1,430 tok/s
[progress] seen 5,000,000/5,567,497 tokens | ~1,424 tok/s
[progress] seen 5,500,000/5,567,497 tokens | ~1,424 tok/s
Epoch done: processed 33,409,326 (center,context) pairs in 3912.1s

=== Epoch 2/3 | base_lr=0.01333 ===
[progress] seen 500,000/5,567,497 tokens | ~1,397 tok/s
[progress] seen 1,000,000/5,567,497 tokens | ~1,429 tok/s
[progress] seen 1,500,000/5,567,497 tokens | ~1,446 tok/s
[progress] seen 2,000,000/5,567,497 tokens | ~1,422

In [3]:
# ================================
# EX4: Embedding utilities & quick evaluation
# ================================
from numpy.linalg import norm

# Use input embeddings as word vectors
E = W_in / (norm(W_in, axis=1, keepdims=True) + 1e-9)

def most_similar(word, topn=10):
    if word not in word2id:
        return []
    wid = word2id[word]
    v = E[wid]
    sims = E @ v
    sims[wid] = -1.0  # exclude self
    top = sims.argsort()[-topn:][::-1]
    return [(id2word[i], float(sims[i])) for i in top]

def analogy(a, b, c, topn=10):
    for w in (a, b, c):
        if w not in word2id:
            return []
    va, vb, vc = E[word2id[a]], E[word2id[b]], E[word2id[c]]
    query = vb - va + vc
    query = query / (norm(query) + 1e-9)
    sims = E @ query
    for w in (a, b, c):
        sims[word2id[w]] = -1.0
    top = sims.argsort()[-topn:][::-1]
    return [(id2word[i], float(sims[i])) for i in top]

def show_neighbors(words, topn=10):
    for w in words:
        if w not in word2id:
            print(f"\n'{w}' not in vocab.")
            continue
        res = most_similar(w, topn=topn)
        print(f"\nNearest to '{w}':")
        for t, sc in res:
            print(f"  {t:>12s}  {sc: .4f}")  # 4 decimals to see spread

probe_words = ["king", "queen", "man", "woman", "city", "music", "science"]
show_neighbors(probe_words, topn=10)

tests = [("man","king","woman"), ("paris","france","rome"), ("good","better","bad")]
for a, b, c in tests:
    res = analogy(a, b, c, topn=5)
    if res:
        print(f"\nAnalogy: {a}:{b} :: {c}:?")
        for t, sc in res[:5]:
            print(f"  {t:>12s}  {sc: .4f}")

# Save vectors (for later use)
np.savez_compressed("sgns_text8_embeddings.npz", W_in=W_in, W_out=W_out, id2word=id2word)
print("\nSaved: sgns_text8_embeddings.npz")



Nearest to 'king':
         whole   0.9957
        during   0.9948
       dragons   0.9945
  supernatural   0.9942
       voltage   0.9939
       friends   0.9939
         those   0.9939
      republic   0.9938
        ethnic   0.9938
           bus   0.9938

Nearest to 'queen':
        ground   0.9972
       supreme   0.9970
     executive   0.9968
       shorter   0.9968
      criteria   0.9964
         teeth   0.9964
         japan   0.9963
        enough   0.9963
          last   0.9963
        formal   0.9962

Nearest to 'man':
        states   0.9938
        plague   0.9938
      criteria   0.9936
         solve   0.9935
        ethnic   0.9935
          been   0.9934
     australia   0.9933
        joined   0.9933
     compounds   0.9932
          land   0.9931

Nearest to 'woman':
   appointment   0.9981
     obviously   0.9980
     manifesto   0.9978
           try   0.9978
     volunteer   0.9977
     microwave   0.9977
     auxiliary   0.9976
       rounded   0.9976
       